
## Simple code for Quantum Random Number Generator

This code should give a random float, generated using qiskit_aer simulator, or real backend.

In [68]:
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_aer import AerSimulator
from qiskit import QuantumCircuit
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import SamplerV2 as Sampler
import sys,time

In [64]:
def generate_random_bits(n:int,n_bits:int,pass_manager,sampler):
    qc = QuantumCircuit(n_bits)
    qc.h(range(n_bits))
    qc.measure_all()
    qc = pass_manager.run(qc)
    job = sampler.run(pubs=[qc],shots=n)

    done= False
    print("Running \n")
    i=1
    while not done:
        if job.done(): done = True
        try:
            status = job.status().name
        except:
            status = job.status()
        if status=="ERROR":
            print("Job ")
            print(status)
            sys.exit(0)
        print("."*i + " "*(3-i), end='\r')
        i=i%3
        i+=1
        time.sleep(2)
    result = job.result()
    dic = result[0].data.meas.get_counts()
    return [key for key, value in dic.items() for _ in range(value)]

def bits_to_float(bits:str):
    random_float = int(bits, 2) / (2 ** len(bits))
    return random_float

In [70]:
backend_name = "ibm_brisbane"
backend = QiskitRuntimeService().backend(name=backend_name)
# Aer
#backend = AerSimulator.from_backend(backend=backend)
#backend_name = backend_name+"_Aer"


### Declare precision of floating point number (number of bits used) and how many numbers to generate 


In [66]:
n_bits=16
n=12

In [71]:
pass_manager = generate_preset_pass_manager(optimization_level=1,backend=backend)
sampler = Sampler(mode=backend)
bits_list = generate_random_bits(n,n_bits,pass_manager,sampler)
result = [bits_to_float(item) for item in bits_list]
print(f"Random {n} floats in range [0,1):\n{result}")

Running 

Random 12 floats in range [0,1):
[0.671295166015625, 0.1432342529296875, 0.8980712890625, 0.8761444091796875, 0.4193572998046875, 0.134490966796875, 0.942352294921875, 0.0821533203125, 0.2520751953125, 0.9700775146484375, 0.84698486328125, 0.4893341064453125]
